# Seq 2 Seq (Sequence to Sequence)

- input sequence 를 가지고 output sequence 를 생성

- 구조

    Encoder : input sequence를 RNN (LSTM, GRU 등) 으로 처리 -> context vector 생성

    Decoder : Encoder가 전달해준 context vector를 가지고 output sequence 생성 -> Encoder와 마찬가지로 RNN (LSTM, GRU 등) 을 통해 번역된 문장 생성

    * Context Vector : input sequence 의 모든 정보를 고정된 길이의 vector로 압축 (= input sequence 요약)

- Teacher Forcing : 기존 RNN - Decoder의 output을 다음 timestep의 Decoder 입력으로 사용 => seq2seq - output 대신 실제 정답(label) 을 다음 timestep의 Decoder 입력으로 사용


- token 종류

    SOS_TOKEN (Start of Sentence Token) : 문장의 시작

    EOS_TOKEN (End of Sentence Token) : 문장의 끝

    PAD_TOKEN (Padding Token) : input sequence 길이를 맞추기 위해 (비어있는 부분을 PAD_TOEN 이 채움)


In [ ]:
import torch
from torch.nn import Module, Embedding, GRU, Dropout, Linear, CrossEntropyLoss, init, utils
from torch.optim import Adam
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoTokenizer

import random
from time import time
import math

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")

elif torch.xpu.is_available():
    device = torch.device("xpu")

elif torch.backends.mps.is_available():
    device = torch.device("mps")

else: 
    device = torch.device("cpu")


device

device(type='cuda')

In [3]:
# dataset load (wmt16의 독일어-영어)
# wmt16 :Workshop on statistical Machine Translation 2016 에서 사용된 기계 번역 데이터셋
# plit="train[:1%]" : 1%만 load (데이터셋이 너무 많아서)
raw_datasets = load_dataset("wmt16", "de-en", split="train[:1%]")

# tokenizer load (학습 속도를 위해 작은 모델 사용) -> huggingface 에서 이미 학습되어있는 모델의 tokenizer 가져옴
# 영어-독일어
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

None
</s>
<pad>


In [4]:
def tokenize_function(examples):
    # English : input sequence (encoder input)
    # [ex['en'] for ex in examples['translation']] : 영어 문장들만 뽑아서 list로 만들고,
    # max_length=64 : 최대 길이를 정하고,
    # truncation=True : 최대 길이를 넘으면 자르고,
    # padding="max_length" : 최대 길이보다 작으면 패딩 추가
    tokenized_en = tokenizer(
        [ex['en'] for ex in examples['translation']],
        max_length=64,
        truncation=True,
        padding="max_length"
    )
    
    # German : output sequen ce (decoder output)
    tokenized_de = tokenizer(
        [ex['de'] for ex in examples['translation']],
        max_length=64,
        truncation=True,
        padding="max_length"
    )
    
    # input_ids : 영어 token들의 id
    # attention_mask : 패딩 부분은 무시 (패딩은 0, 아니면 1)
    # labels : 독일어 token들의 id
    return {
        'input_ids': tokenized_en['input_ids'],
        'attention_mask': tokenized_en['attention_mask'],
        'labels': tokenized_de['input_ids']
    }

In [5]:
# tokenize_function 을 사용해서 datasets 정제
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# pytorch type으로 format 변환
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# train_test_split (90 : 10)
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_data = train_test_split['train']
valid_data = train_test_split['test']

In [6]:
# DataLoader 설정
batch_size = 64
train_iterator = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_iterator = DataLoader(valid_data, batch_size=batch_size)

# 단어장 크기
input_dim = tokenizer.vocab_size
output_dim = tokenizer.vocab_size

# sequence 의 길이를 맞추기 위해 + padding token 은 무시하기 위해
pad_idx = tokenizer.pad_token_id

In [7]:
class Encoder(Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        
        # hidden dimention
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # embedding layer : input_dim 갯수의 단어들을 embedding_dim 의 차원을 가진 vector로 변환
        self.embedding = Embedding(input_dim, embedding_dim)
        
        # rnn layer : GRU 사용 
        # hidden_dim: vector의 크기
        # n_layers : gru layer의 갯수
        # batch_first : input tensor의 첫번재 차원
        self.rnn = GRU(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.dropout = Dropout(dropout)
        
    # src : source sequence (= input sequence)
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        
        # output : Tensor(batch_size, seq_len, hidden_dim) -> embedding 된 input sequence의 GRU 결과 
        # => 지금은 사용되지 않으나, attention model로 발전되면 사용됨
        # hidden : Tensor(n_layers, batch_size, hidden_dim) -> 마지막에 update 된 timestep의 상태 (state)
        output, hidden = self.rnn(embedded) 
        
        # encoder의 최종 hidden tensor를 decoder로 전달
        return hidden

In [8]:
class Decoder(Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = Embedding(output_dim, embedding_dim)
        
        # GRU
        self.rnn = GRU(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        # fully connected layer -> output        
        self.fc_out = Linear(hidden_dim, output_dim)
        
        self.dropout = Dropout(dropout)
        
    # input = [batch size] -> decoder의 현재 시점 입력
    # hidden = [n layers * n directions, batch size, hidden_dim] -> encoder output / 이전 시점 decoder 상태
    def forward(self, input, hidden):

        # [batch size] -> [batch size, 1]로 차원 변경 (rnn 입력 형식에 맞게) -> 각 배치에서, 현재 시점의, 하나의 단어만 입력
        input = input.unsqueeze(1) 
        
        embedded = self.dropout(self.embedding(input))
        
        # output : 현재 시점의 predict 계산 -> 다음 시점의 decoder로 전달
        # hidden : 이전 hidden + 현재 입력 -> 현재 hidden
        output, hidden = self.rnn(embedded, hidden)
        
        # output을 가지고 예측한 결과
        predict = self.fc_out(output.squeeze(1))
        
        # predict = [batch size, output dim] (vocab size)
        return predict, hidden

In [9]:
class Seq2Seq(Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
    
    # src : source sequence (영어)
    # trg : target sequence (독일어)
    # teacher_forcing_ratio = 0.5 : 50% 의 확률로 실제 label을 사용 (나머지 50%는 decoder의 출력-predict- 사용)
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(device)
        
        hidden = self.encoder(src)
        
        # decoder의 첫 입력은 <sos> token
        # trg의 첫 열은 <sos> token이 있는 열로 가정 (hugging face tokenizer 에서 처리됨)
        # trg의 0번째 컬럼 (<sos> token)
        input = trg[:, 0]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            
            # t-1 시점의 predict
            outputs[:, t] = output
            
            # predict 한 결과 token의 id
            top1 = output.argmax(1) 
            
            # teacher forcing 적용 여부 결정
            teacher_force = random.random() < teacher_forcing_ratio
            
            # 다음 input token 설정: 
            # teacher forcing (trg[:, t]) 또는 모델 예측 (top1)
            input = trg[:, t] if teacher_force else top1

        return outputs

In [10]:
embedding_dim = 256
hidden_dim = 1024
n_layers = 3
dropout = 0.3

enc = Encoder(input_dim, embedding_dim, hidden_dim, n_layers, dropout)
dec = Decoder(output_dim, embedding_dim, hidden_dim, n_layers, dropout)

In [11]:
model = Seq2Seq(enc, dec).to(device)

In [12]:
# 가중치 초기화 함수
def init_weights(model):
    for name, param in model.named_parameters():
        if 'weight' in name:
            init.normal_(param.data, mean=0, std=0.01)
        else:
            init.constant_(param.data, 0)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(58101, 256)
    (rnn): GRU(256, 1024, num_layers=3, batch_first=True, dropout=0.3)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(58101, 256)
    (rnn): GRU(256, 1024, num_layers=3, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=1024, out_features=58101, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [13]:
# pad_idx는 손실 계산에서 무시
loss_function = CrossEntropyLoss(ignore_index=pad_idx)

learning_rate = 0.0001
optimizer = Adam(model.parameters(), lr=learning_rate)

In [14]:
# 학습
def train(model, iterator, optimizer, loss_function):
    
    model.train()
    
    epoch_loss = 0
    
    num_batches = len(iterator)
    total_batch_time = 0
    
    for i, batch in enumerate(iterator):
        batch_start_time = time()

        # source sequence (x) - english
        src = batch['input_ids'].to(device)
        # target sequence (y) - germany
        trg = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)

        # output reshape
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        
        # <sos> 제거 후 target sequence reshape
        trg = trg[:, 1:].reshape(-1)
        
        loss = loss_function(output, trg)
        
        loss.backward()
        
        # clip (gradient clipping) : gradient explosion 방지 -> 기울기의 크기가 일정 값 이상으로 커지지 않도록 제한
        utils.clip_grad_norm_(model.parameters(), 0.1)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        batch_end_time = time()
        batch_time = batch_end_time - batch_start_time
        total_batch_time += batch_time

        batch_mins = int(total_batch_time / 60)
        batch_secs = int(total_batch_time% 60)
        
        if i != 0 and i % 100 == 0:
            print(f"\t train batch: {i:3d}/{num_batches} \t loss: {loss.item():.3f} \t batch_time_for_100 : {batch_mins}m {batch_secs}s")
            total_batch_time = 0
        
    return epoch_loss / len(iterator)

In [15]:
# 검증
def evaluate(model, iterator, loss_function):
    
    model.eval()
    
    epoch_loss = 0
    
    num_batches = len(iterator)
    total_batch_time = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            batch_start_time = time()

            src = batch['input_ids'].to(device)
            trg = batch['labels'].to(device)
            
            # teacher forcing을 사용하지 않음
            output = model(src, trg, 0) 

            output_dim = output.shape[-1]
            
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = loss_function(output, trg)

            epoch_loss += loss.item()

            batch_end_time = time()
            batch_time = batch_end_time - batch_start_time
            total_batch_time += batch_time

        batch_mins = int(total_batch_time / 60)
        batch_secs = int(total_batch_time% 60)

        print(f"\t eval batch: {i:3d}/{num_batches} \t loss: {loss.item():.3f} \t batch_time_for_100 : {batch_mins}m {batch_secs}s")

    return epoch_loss / len(iterator)

In [16]:
epochs = 30

total_time = 0

for epoch in range(1,epochs+1):
    print(f"[epoch : {epoch}]")
    start_time = time()
    
    train_loss = train(model, train_iterator, optimizer, loss_function)
    valid_loss = evaluate(model, valid_iterator, loss_function)
    
    # Perplexity (PPL): 낮을수록 좋음. exp(loss) -> 모델이 예측한 확률 분포에 대한 불확실성을 측정
    train_ppl = math.exp(train_loss)
    valid_ppl = math.exp(valid_loss)

    end_time = time()

    epoch_time = end_time - start_time
    total_time += epoch_time
    
    epoch_mins = int((epoch_time) / 60)
    epoch_secs = int((epoch_time) % 60)

    print(f"epoch: {epoch:3d}/{epochs} \t train ppl: {train_ppl:4.3f} val ppl: {valid_ppl:4.3f} \t {epoch_mins}m {epoch_secs}s \n")

print(f"\ntotal time : {total_time}")

[epoch : 1]
	 train batch: 100/640 	 loss: 6.195 	 batch_time_for_100 : 1m 7s
	 train batch: 200/640 	 loss: 6.153 	 batch_time_for_100 : 1m 6s
	 train batch: 300/640 	 loss: 6.192 	 batch_time_for_100 : 1m 6s
	 train batch: 400/640 	 loss: 6.150 	 batch_time_for_100 : 1m 7s
	 train batch: 500/640 	 loss: 6.167 	 batch_time_for_100 : 1m 7s
	 train batch: 600/640 	 loss: 6.156 	 batch_time_for_100 : 1m 5s
	 eval batch:  71/72 	 loss: 6.000 	 batch_time_for_100 : 0m 12s
epoch:   1/30 	 train ppl: 573.636 val ppl: 472.510 	 7m 22s 

[epoch : 2]
	 train batch: 100/640 	 loss: 6.124 	 batch_time_for_100 : 1m 7s
	 train batch: 200/640 	 loss: 6.118 	 batch_time_for_100 : 1m 7s
	 train batch: 300/640 	 loss: 6.095 	 batch_time_for_100 : 1m 7s
	 train batch: 400/640 	 loss: 6.175 	 batch_time_for_100 : 1m 7s
	 train batch: 500/640 	 loss: 6.178 	 batch_time_for_100 : 1m 7s
	 train batch: 600/640 	 loss: 6.277 	 batch_time_for_100 : 1m 6s
	 eval batch:  71/72 	 loss: 6.001 	 batch_time_for_100 

In [17]:
def predict_translation(model, sentence, tokenizer, max_len=64):
    model.eval()

    # pt : pytorch
    tokenized = tokenizer(sentence, return_tensors='pt', truncation=True)

    src = tokenized['input_ids'].to(device)

    # <sos> token (시작)
    sos_token_id = tokenizer.pad_token_id
    
    # decoder의 첫 input token id
    input_token = torch.tensor([sos_token_id], dtype=torch.long, device=device)

    with torch.no_grad():
        hidden = model.encoder(src)

    # 번역된 token 저장
    translated_tokens = []

    # greedy decoding : 한 번에 하나의 토큰을 생성하며 문장을 완성
    for _ in range(max_len):
        
        with torch.no_grad():
            output, hidden = model.decoder(input_token, hidden)

        # 가장 높은 확률을 가진 token의 id
        next_token_id = output.argmax(1).item()

        # 예측된 token이 <eos> 거나 <pad> 이면 종료
        if next_token_id == tokenizer.eos_token_id or next_token_id == tokenizer.pad_token_id:
            # 첫 토큰이 <eos>인 경우 제외
            if len(translated_tokens) > 0:
                break

        # 결과 token 저장
        translated_tokens.append(next_token_id)

        # 현재 token은 다음 시점의 input
        input_token = torch.tensor([next_token_id], dtype=torch.long, device=device)

    # 사람이 읽을 수 있는 단어로 변환
    # skip_special_tokens=True : <eos>, <pad> 같은 token들은 skip
    translated_sentence = tokenizer.decode(translated_tokens, skip_special_tokens=True)
    
    return translated_sentence

In [19]:
test_sentence = "I am happy. Because studing is hard." 

predicted_translation = predict_translation(model, test_sentence, tokenizer)

print(f"input sequence: {test_sentence}")
print(f"output sequence: {predicted_translation}")
# 제대로 학습되었다면 : Ich bin glücklich. Denn das Studium ist anstrengend.

input sequence: I am happy. Because studing is hard.
output sequence: eserdem dert,t,t,t
